Adsorción de hidrógeno

In [1]:
using Plots
using DifferentialEquations
using Interact
using WebIO
using Sundials

- El balance de masa será una ecuación diferencial ordinaria para cada nodo. Estamos asumiendo que no hay difusión, y que la convección tiene un efecto de aumentar la presión en todos los puntos del gas atrapado en el medio poroso.

Para ilustrar el uso de ecuaciones diferenciales algebraicas (DAE), resolveremos el siguiente sistema que representa un modelo de parámetros agrupados.

ODE para la masa de hidrógeno en la fase fluida 

Ec algebraica para cada nodo porque la temperatura es función del radio: $$ n_A = n_0 \exp \left[ {\left(-\frac{RT_i}{{\alpha + \beta T_i}}\right)^m \ln^m\left(\frac{p_0}{p}\right)} \right]$$

$$ f(n_A) =  n_A - n_0 \exp {\left(-\frac{RT}{{\alpha + \beta T}}\right)^m \ln^m\left(\frac{p_0}{p}\right)}  $$

Ecuación diferencial 
$$
\frac{d \rho_{H_2}}{d t} = \frac{\dot{m}_s}{V \varepsilon} - \frac{(1 - \varepsilon) \rho_b M_{H_2} }{\varepsilon} \frac{d n_A}{d t}
$$

$$
\frac{\partial T_i}{\partial t} = \frac{k_{eff} \left( \frac{1}{r} \frac{T_{i+1} - T_{i-1}}{\Delta r} + \frac{T_{i+1} - 2T_i + T_{i-1}}{\Delta r^2} \right) + \varepsilon \frac{dP}{dt} - \alpha \sqrt{\ln \left( \frac{n_o}{n_a} \right)} - (1 - \varepsilon_b) \rho_b M_g \frac{\partial n_a}{\partial t}}
{c_{gb} \rho_g \varepsilon + (1 - \varepsilon) \rho_b c_{pb}}
$$



$n_A$ es u[1], $\rho_{H2}$ es u[2]

In [2]:
# Adsorption 
ε = 0.49 # Porosity 
ρ_b = 517.6 # Activated carbon density / kg m^-3
MW = 2.0159 * 10^-3 # kg/mol  
V = 1.5e-4 # Tank volume
ṁₛ = 2.354e-9 # kg / s

# Equilibrium parameters
R = 8.314 # Gas constant / J / mol / K
α = 3080.0 # Isotherm parameter / K
β = 18.9 # Isotherm parameter / -
p₀ = 1.47e9 # Saturation pressure / Pa
n₀ = 71.6 # Cantidad limite de adsorcion (mol kg-1)
m = 2.0; # magical parameter

In [ ]:
# Aditional parameters for lumped model
ΔH = 3185 # Isostoric heat of adsorption / J / mol 
Aₑ = 0.1277 # Heat transfer area / m^2
h_f = 36 # Heat transfer coefficient / W / m^2 / K 
cₚₛ = 10167 # Heat capacity of the hydrogen / J / kg / K
cₚ_b = 825 # Heat capacity of the activated carbon / J / kg / K
hₛ = 3986.8 # Enthalpy of the hydrogen / J / kg
T_f = 293.15 # Temperature of the exterior / K

In [3]:
# Spherical Tank parameters
n = 101 # Number of points
r = 0.01 # m
Δr = LinRange(0, r, n) # m 
T₀ = 25 + 273.15 # Initial temperature / K
#T = ones(n) * T₀; # Temperature vector / 

298.15

Para mejoras futuras

http://www.coolprop.org/coolprop/wrappers/Julia/

In [8]:
function adsorption(out, du, u, par, t)
    # Unpacking variables
    # T = u[1:n]
    # rho = u[n+1:2n]
    # n_a = u[2n+1:3n]
    nₐ, ρ, p, T = u
    
    # Should be a vector with n-nodes
    out[1] = nₐ - n₀ * exp( - (R*T/(α+β*T))^m * (log(p₀/p))^m ) 
    
    # H2 mass balance
    out[2] = du[2] - ( ṁₛ/(V*ε) - (1-ε)/(ε) * ρ_b * MW * du[1] ) 
    
    # p-rho-T through ideal gas
    out[3] = p - ρ*R*T / MW
    
    # Energy balance
    # Space discretized temperature differential equation
    out[4] = du[4] - (keff * (1/r * (T[i+1] - T[i-1])/Δr + 
    (T[i+1] - 2*T[i] + T[i-1])/Δr^2) + 
    ε * du[3] - α * sqrt(log(n₀/nₐ)) -
     (1 - ε_b) * ρ_b * M_g * dnₐ/dt) / (c_gb * ρ_g * ε + (1 - ε) * ρ_b * c_pb)
    
    # print("\n p_calc", ρ*R*T / MW)
    
    #print("\nout1", out[1])
    #print("\nout2", out[2])
    #print("\nout3", out[3])
    #print("\np = ",p,"\n = rho",ρ, "\nR =",R,"\nT =",T,"\nMW =",MW)
    
end

adsorption (generic function with 1 method)

In [5]:
# Hydrogen at 1 atm at the beginning of the charging period
pᵢ = 1e5 # Pa
ρ₀ = pᵢ/( (R/MW) *T)

# Consistent initial conditions for adsorbed mass in hydrogen
# this formula MUST be used if no mass transfer coefficient is used
# (instantaneous equilibrium)
na₀ = n₀ * exp( - (R*T/(α+β*T))^m * (log(p₀/pᵢ))^m ) 

u₀ = [na₀, ρ₀ , pᵢ, T₀]

# Initial estimates of temporal variables
du₀ = [1e-4, 1e-4, 1e-4]

# Tiempo de integración
tspan = (0.0, 3600*100)

# Variables that appear explicitely as derivatives du
differential_vars = [true, true, true, true]

# Differential algebraic problem for sundials
prob = DAEProblem(adsorption,du₀,u₀,tspan,differential_vars=differential_vars)

sol_2= solve(prob,IDA());

UndefVarError: UndefVarError: T not defined

In [6]:
plot(sol_2, layout = (3,1))

UndefVarError: UndefVarError: sol_2 not defined